# **Robot Path Planning with A* Search Algorithm**
## **Problem**
Find the shortest path from a start position to the closest goal position on a grid with obstacles. The robot can move up, down, left, or right (no diagonals). I will use the A* search algorithm with an admissible heuristic.


# **A* algorithm**

 A* algorithm is an informed search algorithm, leverages a heuristic function to guide its search towards the goal. This heuristic function estimates the cost of reaching the goal from a given node, allowing the algorithm to prioritize promising paths.  

The A* algorithm's efficiency comes from its smart evaluation of paths using three key components: g(n), h(n), and f(n). These components work together to guide the search process toward the most promising paths.


> Add blockquote



# **Step-by-Step Explanation**

- Input Reading: The program reads the grid dimensions, start position, goal positions, and blocked cells.

- Admissible Heuristic: For multiple goals, we need a heuristic that underestimates the distance to the closest goal.

- Heuristic Function: For any cell (r, c), it calculates the Manhattan distance to all goals and returns the minimum value.

- *A Search**:
-- Uses a priority queue (min-heap) to explore nodes with the lowest f(n) = g(n) + h(n) first.
-- g(n) is the actual cost from start to current node.
-- h(n) is the heuristic estimate to the closest goal.
- For each move, checks if the new position is valid (within bounds and not blocked).
- Track the path: Use a parent dictionary to reconstruct the path once a goal is found.
- Termination: When a goal is reached, returns the path. If the queue is exhausted without finding a goal, returns an empty path (though the problem assumes a path exists).

# **Step by Step A* Search Algorithm**


In [2]:
# Importing required library

import heapq

# Manhattan distance function

def manhattan_dist(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# A* search function

def a_star_search(rows, cols, start, goals, blocked):

# Initialization
    directions = [(-1, 0, 'U'), (1, 0, 'D'), (0, -1, 'L'), (0, 1, 'R')] # Directions for movement: Up, Down, Left, Right. Each includes a direction vector and a move label

    grid = [[' ' for _ in range(cols)] for _ in range(rows)] #Initializes the grid with empty spaces
# Marking blocked, goal, and start cells

    for r, c in blocked:
        grid[r][c] = 'X' #Marks blocked cells with 'X'.
    for r, c in goals:
        grid[r][c] = 'G'
    sr, sc = start
    grid[sr][sc] = 'S'
#Preparing for search
## Ensures that start, goals, and blocked are in tuple form (suitable for hashing)
## Converts blocked to a set for fast lookup.

    start = tuple(start)
    goals = [tuple(g) for g in goals]
    blocked_set = set(blocked)
# Open set and g_score
## Pushes the starting node into the priority queue open_set:
## f = g + h, where: g = 0 (cost from start), h = min(manhattan_dist to any goal)
## came_from stores the path (parent node and move direction)
## g_score keeps the lowest cost to reach each node.

    open_set = []
    heapq.heappush(open_set, (0 + min(manhattan_dist(start, g) for g in goals), 0, start))
    came_from = {}
    g_score = {start: 0}

# Main A* Loop

    while open_set:
        f, cost, current = heapq.heappop(open_set) #Pops the node with the lowest f value

        if current in goals:    #If the current node is a goal, reconstruct the path
# Path reconstruction
## Walks back from the goal to the start, building the path using came_from
            path = []
            cur = current
            while cur != start:
                prev, move = came_from[cur]
                path.append((prev, cur, move))
                cur = prev
            path.reverse()
            for _, (r, c), move in path:     #Marks the path on the grid with the move directions
                if grid[r][c] == ' ':
                    grid[r][c] = move
            return ''.join(move for _, _, move in path), grid #Returns the move sequence and the updated grid
# Exploring neighbors

        for dr, dc, move in directions:     #Tries moving in each direction
            nr, nc = current[0] + dr, current[1] + dc
            neighbor = (nr, nc)
            if 0 <= nr < rows and 0 <= nc < cols and neighbor not in blocked_set:  #The neighbor is within bounds and not blocked
                new_cost = cost + 1 #Calculates new cost to reach the neighbor. If it’s better than any previously known path to this neighbor, then update it
                if neighbor not in g_score or new_cost < g_score[neighbor]:
                    g_score[neighbor] = new_cost  #Adds the neighbor to the open set with updated cost and priority, and records where it came from
                    priority = new_cost + min(manhattan_dist(neighbor, g) for g in goals)
                    heapq.heappush(open_set, (priority, new_cost, neighbor))
                    came_from[neighbor] = (current, move)
# If No Path Found
    return "No path found", grid #Returns a message and the grid if no goal is reachable
# Grid printing function

def print_grid(grid):  #Prints the grid with coordinates
    print("   " + " ".join(f"{i}" for i in range(len(grid[0]))))
    print("  +" + "--"*len(grid[0]) + "+")
    for i, row in enumerate(grid):
        print(f"{i} |" + " ".join(row) + "|")
    print("  +" + "--"*len(grid[0]) + "+")

#Running a test case
## This is a helper function to test the pathfinding:Displays input, Calls a_star_search, Shows the path and final grid

def run_test_case(case_id, rows, cols, start, goals, blocked):
    print(f"\n🧪 Test Case {case_id}:")
    print(f"Grid Size: {rows}x{cols}")
    print(f"Start: {start}")
    print(f"Goals: {goals}")
    print(f"Blocked: {blocked}")
    path, grid = a_star_search(rows, cols, start, goals, blocked)
    print("🟩 Output Path:", path)
    print("🗺️ Final Grid:")
    print_grid(grid)

# --- Test Case 1: Provided Input ---
run_test_case(1,
    5, 6,
    (1, 0),  # 2 1
    [(1, 5), (3, 2), (4, 4)],  # 2 6, 4 3, 5 5
    [(1,1), (3,1), (1,2), (2,2), (2,4), (3,4)]  # blocked
)

# --- Test Case 2: New layout ---
run_test_case(2,
    5, 6,
    (0, 0),  # 1 1
    [(0, 5), (3, 3), (4, 4)],  # 1 6, 4 4, 5 5
    [(0,1), (1,1), (2,1), (3,1), (3,2), (3,4)]
)

# --- Test Case 3: Dense block with limited route ---
run_test_case(3,
    5, 6,
    (2, 0),  # 3 1
    [(0, 5), (4, 2), (4, 5)],  # 1 6, 5 3, 5 6
    [(1,0), (1,1), (2,1), (2,2), (3,3), (3,4)]
)



🧪 Test Case 1:
Grid Size: 5x6
Start: (1, 0)
Goals: [(1, 5), (3, 2), (4, 4)]
Blocked: [(1, 1), (3, 1), (1, 2), (2, 2), (2, 4), (3, 4)]
🟩 Output Path: DDDRRU
🗺️ Final Grid:
   0 1 2 3 4 5
  +------------+
0 |           |
1 |S X X     G|
2 |D   X   X  |
3 |D X G   X  |
4 |D R R   G  |
  +------------+

🧪 Test Case 2:
Grid Size: 5x6
Start: (0, 0)
Goals: [(0, 5), (3, 3), (4, 4)]
Blocked: [(0, 1), (1, 1), (2, 1), (3, 1), (3, 2), (3, 4)]
🟩 Output Path: DDDDRRRU
🗺️ Final Grid:
   0 1 2 3 4 5
  +------------+
0 |S X       G|
1 |D X        |
2 |D X        |
3 |D X X G X  |
4 |D R R R G  |
  +------------+

🧪 Test Case 3:
Grid Size: 5x6
Start: (2, 0)
Goals: [(0, 5), (4, 2), (4, 5)]
Blocked: [(1, 0), (1, 1), (2, 1), (2, 2), (3, 3), (3, 4)]
🟩 Output Path: DRRD
🗺️ Final Grid:
   0 1 2 3 4 5
  +------------+
0 |          G|
1 |X X        |
2 |S X X      |
3 |D R R X X  |
4 |    G     G|
  +------------+


# **Save 3 Sample as a 3 Different Input and Output Files**
### Add some of the new codes at the last
### File will save into *the* current working folder

In [ ]:
import heapq

def manhattan_dist(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(rows, cols, start, goals, blocked):
    directions = [(-1, 0, 'U'), (1, 0, 'D'), (0, -1, 'L'), (0, 1, 'R')]
    grid = [[' ' for _ in range(cols)] for _ in range(rows)]

    for r, c in blocked:
        grid[r][c] = 'X'
    for r, c in goals:
        grid[r][c] = 'G'
    sr, sc = start
    grid[sr][sc] = 'S'

    start = tuple(start)
    goals = [tuple(g) for g in goals]
    blocked_set = set(blocked)

    open_set = []
    heapq.heappush(open_set, (0 + min(manhattan_dist(start, g) for g in goals), 0, start))
    came_from = {}
    g_score = {start: 0}

    while open_set:
        f, cost, current = heapq.heappop(open_set)

        if current in goals:
            path = []
            cur = current
            while cur != start:
                prev, move = came_from[cur]
                path.append((prev, cur, move))
                cur = prev
            path.reverse()
            for _, (r, c), move in path:
                if grid[r][c] == ' ':
                    grid[r][c] = move
            return ''.join(move for _, _, move in path), grid

        for dr, dc, move in directions:
            nr, nc = current[0] + dr, current[1] + dc
            neighbor = (nr, nc)
            if 0 <= nr < rows and 0 <= nc < cols and neighbor not in blocked_set:
                new_cost = cost + 1
                if neighbor not in g_score or new_cost < g_score[neighbor]:
                    g_score[neighbor] = new_cost
                    priority = new_cost + min(manhattan_dist(neighbor, g) for g in goals)
                    heapq.heappush(open_set, (priority, new_cost, neighbor))
                    came_from[neighbor] = (current, move)
    return "No path found", grid

def format_grid(grid):
    output = []
    output.append("   " + " ".join(f"{i}" for i in range(len(grid[0]))))
    output.append("  +" + "--"*len(grid[0]) + "+")
    for i, row in enumerate(grid):
        output.append(f"{i} |" + " ".join(row) + "|")
    output.append("  +" + "--"*len(grid[0]) + "+")
    return "\n".join(output)

def save_test_case(case_id, rows, cols, start, goals, blocked):
    path, final_grid = a_star_search(rows, cols, start, goals, blocked)

    # Input file content
    input_data = f"{rows} {cols}\n" \
                 f"{start[0]+1} {start[1]+1}\n" \
                 f"{len(goals)}\n" + \
                 "\n".join(f"{r+1} {c+1}" for r, c in goals) + "\n" \
                 f"{len(blocked)}\n" + \
                 "\n".join(f"{r+1} {c+1}" for r, c in blocked)

    # Output file content
    output_data = f"Path: {path}\n\nGrid:\n{format_grid(final_grid)}"

    with open(f"input{case_id}.txt", "w") as f:
        f.write(input_data)
    with open(f"output{case_id}.txt", "w") as f:
        f.write(output_data)

# Define 3 test cases
test_cases = [
    (1, 5, 6, (1, 0), [(1, 5), (3, 2), (4, 4)], [(1,1), (3,1), (1,2), (2,2), (2,4), (3,4)]),
    (2, 5, 6, (0, 0), [(0, 5), (3, 3), (4, 4)], [(0,1), (1,1), (2,1), (3,1), (3,2), (3,4)]),
    (3, 5, 6, (2, 0), [(0, 5), (4, 2), (4, 5)], [(1,0), (1,1), (2,1), (2,2), (3,3), (3,4)]),
]

# Run and save all test cases
for case in test_cases:
    save_test_case(*case)


# **Generate and Save Visualization Images of the Grid**
### Computed A* path for your 3 test cases
### Using matplotlib to visualize


In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_grid(grid, case_id):
    rows, cols = len(grid), len(grid[0])
    fig, ax = plt.subplots(figsize=(cols, rows))

    # Draw grid
    for r in range(rows):
        for c in range(cols):
            char = grid[r][c]
            color = "white"
            if char == 'X':
                color = "black"
            elif char == 'S':
                color = "green"
            elif char == 'G':
                color = "red"
            elif char in ['U', 'D', 'L', 'R']:
                color = "lightblue"

            rect = patches.Rectangle((c, rows - r - 1), 1, 1, facecolor=color, edgecolor='gray')
            ax.add_patch(rect)

            if char in ['S', 'G']:
                ax.text(c + 0.5, rows - r - 0.5, char, ha='center', va='center', fontsize=12, color='white' if color != "white" else "black")
            elif char in ['U', 'D', 'L', 'R']:
                ax.text(c + 0.5, rows - r - 0.5, char, ha='center', va='center', fontsize=10)

    ax.set_xlim(0, cols)
    ax.set_ylim(0, rows)
    ax.set_xticks(range(cols))
    ax.set_yticks(range(rows))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(False)
    ax.set_aspect('equal')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"grid_case_{case_id}.png")
    plt.close()

# Visualize each test case
for case in test_cases:
    case_id = case[0]
    _, grid = a_star_search(*case[1:])
    visualize_grid(grid, case_id)

# **Output**

### grid_case_1.png
### grid_case_2.png
### grid_case_3.png
## **Each image shows:**
### Black cells for obstacles (X)
### Green cell for start (S)
### Red cells for goals (G)
### Light blue path with directions (U, D, L, R)